Google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

path_to_drive = "/content/drive/MyDrive/Colab Notebooks"

Mounted at /content/drive


## spec

In [ ]:
!nvidia-smi

Fri Sep  3 01:46:12 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8    29W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Get GitHub repository files and install python packages

In [ ]:
!git clone https://github.com/Team-M1/badwords-classifier-train -b WJ
%cd badwords-classifier-train
!pip install -r requirements.txt

Cloning into 'badwords-classifier-train'...
remote: Enumerating objects: 189, done.
remote: Counting objects: 100% (189/189), done.
remote: Compressing objects: 100% (168/168), done.
remote: Total 189 (delta 107), reused 102 (delta 20), pack-reused 0
Receiving objects: 100% (189/189), 2.79 MiB | 3.06 MiB/s, done.
Resolving deltas: 100% (107/107), done.
/content/badwords-classifier-train
     |████████████████████████████████| 264 kB 13.4 MB/s 
     |████████████████████████████████| 282 kB 49.8 MB/s 
     |████████████████████████████████| 2.8 MB 44.1 MB/s 
     |████████████████████████████████| 919 kB 44.8 MB/s 
     |████████████████████████████████| 243 kB 53.1 MB/s 
     |████████████████████████████████| 50 kB 7.0 MB/s 
     |████████████████████████████████| 119 kB 55.9 MB/s 
     |████████████████████████████████| 636 kB 48.5 MB/s 
     |████████████████████████████████| 3.3 MB 41.2 MB/s 
     |████████████████████████████████| 895 kB 51.9 MB/s 
     |██████████████████████████

## model and tokenizer



In [ ]:
model_config = {
    "num_labels": 3,
    "id2label": {0: 0, 1: 1, 2: 2},
    "label2id": {0: 0, 1: 1, 2: 2}
}

In [ ]:
from transformers import ElectraForSequenceClassification, ElectraTokenizer

model = ElectraForSequenceClassification.from_pretrained("monologg/koelectra-small-v3-discriminator", **model_config)
tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-small-v3-discriminator")

Downloading:   0%|          | 0.00/458 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/56.6M [00:00<?, ?B/s]

Some weights of the model checkpoint at monologg/koelectra-small-v3-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.weight']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-small-v3-discriminator and are newly initialized

Downloading:   0%|          | 0.00/263k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/61.0 [00:00<?, ?B/s]

## datasets

In [ ]:
from data_loader import get_data_loaders

train_data, val_data, test_data = get_data_loaders(tokenizer, return_loader=False)

Using custom data configuration default-68900ace4d447dfd


0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-68900ace4d447dfd/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff. Subsequent calls will reuse this data.


  0%|          | 0/40242 [00:00<?, ?ex/s]

  0%|          | 0/4472 [00:00<?, ?ex/s]

  0%|          | 0/5000 [00:00<?, ?ex/s]

## metric

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    f1 = f1_score(y_true=labels, y_pred=pred, average="macro")

    return {"accuracy": accuracy, "f1": f1}

## Do Train

In [ ]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
from transformers import set_seed

set_seed(42)

In [ ]:
from transformers import TrainingArguments

args = TrainingArguments(
    output_dir=f'{path_to_drive}/koelectra-small-v3-discriminator-1',
    seed=42,
    num_train_epochs=200,
    learning_rate=1e-4,
    weight_decay=0.0,
    gradient_accumulation_steps=1,
    adam_epsilon=1e-8,
    max_grad_norm=1.0,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,

    # checkpoint
    overwrite_output_dir=True,
    save_strategy='steps',
    save_steps=1000,

    # evaluation
    evaluation_strategy='steps',
    eval_steps=1000,
    metric_for_best_model="f1",

    # early stopping
    load_best_model_at_end=True
)

In [ ]:
from transformers import EarlyStoppingCallback
from trainer import ImbalancedSamplerTrainer

trainer = ImbalancedSamplerTrainer(
    model=model,
    args=args,
    train_dataset=train_data,
    eval_dataset=val_data,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=10)],
)

In [ ]:
trainer.train()

***** Running training *****
  Num examples = 40242
  Num Epochs = 200
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 251600


Step,Training Loss,Validation Loss,Accuracy,F1
1000,0.331300,0.358022,0.866055,0.674720
2000,0.212800,0.397397,0.870304,0.666397
3000,0.160900,0.340801,0.904070,0.705312
4000,0.127800,0.441737,0.880367,0.665402
5000,0.125400,0.329157,0.919052,0.721987
6000,0.105500,0.402963,0.904517,0.710867
7000,0.096200,0.406034,0.909436,0.707301
8000,0.085400,0.411341,0.908542,0.717175
9000,0.078400,0.397584,0.928667,0.743614
10000,0.077900,0.499014,0.902281,0.700300


***** Running Evaluation *****
  Num examples = 4472
  Batch size = 64
Saving model checkpoint to /content/drive/MyDrive/Colab Notebooks/koelectra-small-v3-discriminator-1/checkpoint-1000
Configuration saved in /content/drive/MyDrive/Colab Notebooks/koelectra-small-v3-discriminator-1/checkpoint-1000/config.json
Model weights saved in /content/drive/MyDrive/Colab Notebooks/koelectra-small-v3-discriminator-1/checkpoint-1000/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 4472
  Batch size = 64
Saving model checkpoint to /content/drive/MyDrive/Colab Notebooks/koelectra-small-v3-discriminator-1/checkpoint-2000
Configuration saved in /content/drive/MyDrive/Colab Notebooks/koelectra-small-v3-discriminator-1/checkpoint-2000/config.json
Model weights saved in /content/drive/MyDrive/Colab Notebooks/koelectra-small-v3-discriminator-1/checkpoint-2000/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 4472
  Batch size = 64
Saving model checkpoint to /content/drive

In [ ]:
# train from checkpoint
# checkpoint = `checkpoint-1000`
# trainer.train(resume_from_checkpoint=f'{path_to_drive}/koelectra-small-v3-discriminator-1/{checkpoint}')